In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = None

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

In [2]:
df = pd.read_csv('../FILES/attacks_limpio_1.csv', index_col = 0)
df.head(2)

,case_number,year,type,country,activity,age,species_,month,fatal,gender
0,2018.06.25,2018,Boating,usa,Paddling,57,White shark,Jun,n,F
1,2018.06.18,2018,Unprovoked,usa,Standing,11,NaN,Jun,n,F


1- Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

 - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.
 
 - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

Imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'

 tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo 
 White shark. 
 Un patrón que podriamos usar es: 

patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

 de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las 
 otras 4 especies que queremos "encontrar". 

In [4]:
# comm en el anterior, hacemos copia 
df_copy = df.copy()
df_copy.head(2)

,case_number,year,type,country,activity,age,species_,month,fatal,gender
0,2018.06.25,2018,Boating,usa,Paddling,57,White shark,Jun,n,F
1,2018.06.18,2018,Unprovoked,usa,Standing,11,NaN,Jun,n,F


In [5]:
# patron_blanco = r".*[Ww](hite|HITE).*"
White = r".*[Ww](hite|HITE).*"
Tiger = r".*[Tt](iger|IGER).*"
Grey = r".*[Gg](ray|RAY).*"
Lemon = r".*[Ll](emon|EMON).*"
Bull = r".*[Bb](ull|ULL).*"

In [7]:
df_copy['species_'].isnull().sum()

2831

In [8]:
# extraer por los patrones definidos
def rename(x):
    try:   
        if len(re.findall(White, str(x))) > 0:
            return 'White'
        elif len(re.findall(Tiger, str(x))) > 0:
            return 'Tiger'
        elif len(re.findall(Grey, str(x))) > 0:
            return 'Grey'
        elif len(re.findall(Lemon, str(x))) > 0:
            return 'Lemon'
        elif len(re.findall(Bull, str(x))) > 0:
            return 'Bull'
        else:
            return 'Unknown' 
    except:
            return 'Unknown'
    

In [9]:
# modificar valores
df_copy['species_'] = df_copy['species_'].apply(rename)

In [10]:
# comprobamos los cambios, value_counts() 
df_copy['species_'].value_counts()

Unknown    5122
White       666
Tiger       282
Bull        172
Lemon        37
Grey          5
Name: species_, dtype: int64

2- Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
 - Edad en formato string
 - Edades separadas por &, or, to, >
 - Edades con ?
 - Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
    -  Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
   - Puede que os salga un error similar a este:
                     - TypeError: expected string or bytes-like object
   - Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
                     -  df['nombre_columna'] = df['nombre_columna'].astype(str)
    - Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
    

In [11]:
#para evitar que salga el error mencionado:
df_copy['age'] = df_copy['age'].astype(str)


In [12]:
df_copy['age'].dtype

dtype('O')

In [14]:
# patrón para sacar digitos (d: digit[0-9] +:one or more times) = \d+

In [14]:
def age_clean(i):
    try:
        age = (re.findall(r'\d+', str(i)))
        return age
    except:
        return np.nan

In [15]:
df_copy['age'] = df_copy['age'].apply(age_clean)

In [16]:
df_copy.head(2)

,case_number,year,type,country,activity,age,species_,month,fatal,gender
0,2018.06.25,2018,Boating,usa,Paddling,[57],White,Jun,n,F
1,2018.06.18,2018,Unprovoked,usa,Standing,[11],Unknown,Jun,n,F


In [19]:
#Como con el findall nos devuelve una lista en la columna age, hacemos un explode para quitarlo
df_copy = df_copy.explode('age')
df_copy.head(2)

,case_number,year,type,country,activity,age,species_,month,fatal,gender
0,2018.06.25,2018,Boating,usa,Paddling,57,White,Jun,n,F
1,2018.06.18,2018,Unprovoked,usa,Standing,11,Unknown,Jun,n,F


In [18]:
# comprobamos los cambios
df_copy['age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '13', '42', '3', '8', '50',
       '16', '82', '73', '68', '51', '39', '58', '47', '61', '65', '36',
       '66', '43', '9', '72', '59', '6', '27', '64', '23', '71', '44',
       '62', '63', '70', '53', '77', '74', '5', '86', '84', '75', '87',
       '67', '1', '2', '81', '78'], dtype=object)

- Por último cambiad el tipo de la columna de string a integer.

In [20]:
df_copy['age'] = df_copy['age'].astype('Int64')


In [21]:
# comprobamos
df_copy['age'].dtype

Int64Dtype()

In [22]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6328 entries, 0 to 6283
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   case_number  6327 non-null   object
 1   year         6328 non-null   int64 
 2   type         6324 non-null   object
 3   country      6278 non-null   object
 4   activity     5784 non-null   object
 5   age          3478 non-null   Int64 
 6   species_     6328 non-null   object
 7   month        5414 non-null   object
 8   fatal        5700 non-null   object
 9   gender       6328 non-null   object
dtypes: Int64(1), int64(1), object(8)
memory usage: 550.0+ KB


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio

In [23]:
df_copy.to_csv('../FILES/attacks_limpio_2.csv')